In [ ]:
!git clone https://github.com/HoloClean/holoclean.git

Cloning into 'holoclean'...
remote: Enumerating objects: 2267, done.
remote: Total 2267 (delta 0), reused 0 (delta 0), pack-reused 2267
Receiving objects: 100% (2267/2267), 8.28 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (1500/1500), done.


In [ ]:
pip install -r /content/CleanML/requirements.txt

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 29.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4a59b05e893bf36e22b861634e4c28f9b21ef04f17cfcae3f6148cccdb86118b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/chu-data-lab/CleanML.git

fatal: destination path 'CleanML' already exists and is not an empty directory.


In [ ]:
!pip install utils

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
!git clone https://github.com/cleanlab/cleanlab.git

Cloning into 'cleanlab'...
remote: Enumerating objects: 4017, done.
remote: Total 4017 (delta 0), reused 0 (delta 0), pack-reused 4017
Receiving objects: 100% (4017/4017), 1.12 MiB | 5.63 MiB/s, done.
Resolving deltas: 100% (2725/2725), done.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import sys
import utils

###Outlier-Detection: SD,IQR,IF

In [ ]:
def SD(x, nstd=3.0):
    # Standard Deviaiton Method (Univariate)
    mean, std = np.mean(x), np.std(x)
    cut_off = std * nstd
    lower, upper = mean - cut_off, mean + cut_off
    return lambda y: (y > upper) | (y < lower)

def IQR(x, k=1.5):
    # Interquartile Range (Univariate)
    q25, q75 = np.percentile(x, 25), np.percentile(x, 75)
    iqr = q75 - q25
    cut_off = iqr * k
    lower, upper = q25 - cut_off, q75 + cut_off
    return lambda y: (y > upper) | (y < lower)

def IF(x, contamination=0.01):
    # Isolation Forest (Univariate)
    IF = IsolationForest(contamination=contamination)
    IF.fit(x.reshape(-1, 1))
    return lambda y: (IF.predict(y.reshape(-1, 1)) == -1)

###Outlier-Repair: Mean,Median,Mode

In [ ]:
  
    def fit(self, dataset, df):
        num_df = df.select_dtypes(include='number')
        cat_df = df.select_dtypes(exclude='number')
        X = num_df.values
        m = X.shape[1]

        self.detectors = []
        for i in range(m):
            x = X[:, i]
            detector = self.detect_fn(x, **self.kwargs)
            self.detectors.append(detector)

        ind = self.detect(df)
        df_copy = df.copy()
        df_copy[ind] = np.nan
        self.repairer.fit(dataset, df_copy)
        self.is_fit = True

    def detect(self, df):
        num_df = df.select_dtypes(include='number')
        cat_df = df.select_dtypes(exclude='number')
        X = num_df.values
        m = X.shape[1]

        ind_num = np.zeros_like(num_df).astype('bool')
        ind_cat = np.zeros_like(cat_df).astype('bool')
        for i in range(m):
            x = X[:, i]
            detector = self.detectors[i]
            is_outlier = detector(x)
            ind_num[:, i] = is_outlier

        ind_num = pd.DataFrame(ind_num, columns=num_df.columns)
        ind_cat = pd.DataFrame(ind_cat, columns=cat_df.columns)
        ind = pd.concat([ind_num, ind_cat], axis=1).reindex(columns=df.columns)
        return ind

    def repair(self, df, ind):
        df_copy = df.copy()
        df_copy[ind] = np.nan
        df_clean, _ = self.repairer.clean_df(df_copy)
        return df_clean

    def clean_df(self, df, ignore=None):
        if not self.is_fit:
            print("Must fit before clean")
            sys.exit()
        ind = self.detect(df)
        if ignore is not None:
            ind.loc[:, ignore] = False
        df_clean = self.repair(df, ind)
        return df_clean, ind

    def clean(self, dirty_train, dirty_test):
        clean_train, indicator_train = self.clean_df(dirty_train)
        clean_test, indicator_test = self.clean_df(dirty_test)
        return clean_train, indicator_train, clean_test, indicator_test

###Outlier-Repair: Holoclean

In [ ]:
    def fit(self, dataset, df):
        clean_raw_path = utils.get_dir(dataset, 'raw', '/content/airbnb_constraints.txt')
        index_train_path = utils.get_dir(dataset, 'raw', 'idx_train.csv')
        index_test_path = utils.get_dir(dataset, 'raw', 'idx_test.csv')

        index_train = pd.read_csv(index_train_path).values.reshape(-1)
        index_test = pd.read_csv(index_test_path).values.reshape(-1)
        clean_raw = pd.read_csv(clean_raw_path)

        if 'missing_values' in dataset['error_types']:
            dirty_train = pd.read_csv(utils.get_dir(dataset, 'raw', 'dirty_train.csv'))
            dirty_test = pd.read_csv(utils.get_dir(dataset, 'raw', 'dirty_test.csv'))
            raw = pd.read_csv(utils.get_dir(dataset, 'raw', 'raw.csv'))
            raw_mv_rows = raw.isnull().values.any(axis=1)
            train_mv_rows = dirty_train.isnull().values.any(axis=1)
            test_mv_rows = dirty_test.isnull().values.any(axis=1)

            old_index = np.arange(len(raw))[raw_mv_rows == False]
            new_index = np.arange(len(raw) - sum(raw_mv_rows))
            index_map = {}

            for o, n in zip(old_index, new_index):
                index_map[o] = n

            index_train_no_mv = index_train[train_mv_rows == False]
            index_test_no_mv = index_test[test_mv_rows == False]

            index_train = [index_map[i] for i in index_train_no_mv]
            index_test = [index_map[i] for i in index_test_no_mv]

        self.clean_train = clean_raw.iloc[index_train, :]
        self.clean_test = clean_raw.iloc[index_test, :]

    def clean(self, dirty_train, dirty_test):
        indicator_train = pd.DataFrame(dirty_train.values != self.clean_train.values, columns=dirty_train.columns)
        indicator_test = pd.DataFrame(dirty_test.values != self.clean_test.values, columns=dirty_train.columns)

        clean_train = self.clean_train
        clean_test =self.clean_test
        return clean_train, indicator_train, clean_test

###Missing values-Repair: Mean_Dummy,Median_Dummy,Mode_Dummy

In [ ]:
    def detect(self, df):
        return df.isnull()

    def fit(self, dataset, df):
        if self.method == 'impute':
            num_method = self.kwargs['num']
            cat_method = self.kwargs['cat']
            num_df = df.select_dtypes(include='number')
            cat_df = df.select_dtypes(exclude='number')
            if num_method == "mean":
                num_imp = num_df.mean()
            if num_method == "median":
                num_imp = num_df.median()
            if num_method == "mode":
                num_imp = num_df.mode().iloc[0]

            if cat_method == "mode":
                cat_imp = cat_df.mode().iloc[0]
            if cat_method == "dummy":
                cat_imp = ['missing'] * len(cat_df.columns)
                cat_imp = pd.Series(cat_imp, index=cat_df.columns)
            self.impute = pd.concat([num_imp, cat_imp], axis=0)
        self.is_fit = True

    def repair(self, df):
        if self.method == 'delete':
            df_clean = df.dropna()

        if self.method == 'impute':
            df_clean = df.fillna(value=self.impute)
        return df_clean

    def clean_df(self, df):
        if not self.is_fit:
            print('Must fit before clean.')
            sys.exit()
        mv_mat = self.detect(df)
        df_clean = self.repair(df)
        return df_clean, mv_mat

    def clean(self, dirty_train, dirty_test):
        clean_train, indicator_train = self.clean_df(dirty_train)
        clean_test, indicator_test = self.clean_df(dirty_test)
        return clean_train, indicator_train, clean_test, indicator_test

###Missing Values-Repair: Holoclean

In [ ]:
    def detect(self, df):
        return df.isnull()

    def fit(self, dataset, df):
        clean_raw_path = utils.get_dir(dataset, 'raw', '/content/titanic_constraints.txt')
        clean_raw = pd.read_csv(clean_raw_path)

        index_train_path = utils.get_dir(dataset, 'raw', 'idx_train.csv')
        index_test_path = utils.get_dir(dataset, 'raw', 'idx_test.csv')
        index_train = pd.read_csv(index_train_path).values.reshape(-1)
        index_test = pd.read_csv(index_test_path).values.reshape(-1)

        self.clean_train = clean_raw.iloc[index_train, :]
        self.clean_test = clean_raw.iloc[index_test, :]

    def clean(self, dirty_train, dirty_test):
        indicator_train = self.detect(dirty_train)
        indicator_test = self.detect(dirty_test)

        clean_train = self.clean_train
        clean_test =self.clean_test
        return clean_train, indicator_train, clean_test, indicator_test

###Duplicates-Detect&Repair

In [ ]:
    def fit(self, dataset, df):
        self.keys = dataset['key_columns']
    
    def detect(self, df, keys):
        key_col = pd.DataFrame(df, columns=keys)
        is_dup = key_col.duplicated(keep='first')
        is_dup = pd.DataFrame(is_dup, columns=['is_dup'])
        return is_dup

    def repair(self, df, is_dup):
        not_dup = (is_dup.values == False)
        df_clean = df[not_dup]
        return df_clean

    def clean_df(self, df):
        is_dup = self.detect(df, self.keys)
        df_clean = self.repair(df, is_dup)
        return df_clean, is_dup

    def clean(self, dirty_train, dirty_test):
        clean_train, indicator_train = self.clean_df(dirty_train)
        clean_test, indicator_test = self.clean_df(dirty_test)
        return clean_train, indicator_train, clean_test, indicator_test

In [ ]:
def cleanedDf(dataset):
    # create saving folder
    major_save_dir = utils.makedirs([config.data_dir, dataset["data_dir"] + "_major", 'raw'])
    minor_save_dir = utils.makedirs([config.data_dir, dataset["data_dir"] + "_minor", 'raw'])
    uniform_save_dir = utils.makedirs([config.data_dir, dataset["data_dir"] + "_uniform", 'raw'])

    # load clean data
    clean_path = utils.get_dir(dataset, 'raw', 'raw.csv')
    clean = utils.load_df(dataset, clean_path)
    clean = clean.dropna().reset_index(drop=True)

    major_clean_path = os.path.join(major_save_dir, 'mv_clean_raw.csv')
    minor_clean_path = os.path.join(minor_save_dir, 'out_clean_raw.csv')
    uniform_clean_path = os.path.join(uniform_save_dir, 'inconsist_clean_raw.csv')
    clean.to_csv(major_clean_path, index=False)
    clean.to_csv(minor_clean_path, index=False)
    clean.to_csv(uniform_clean_path, index=False)

    label = dataset['label']

    # uniform flip
    uniform = uniform_class_noise(clean, label)
    # pairwise flip
    major, minor = pairwise_class_noise(clean, label)

    major_raw_path = os.path.join(major_save_dir, 'raw.csv')
    minor_raw_path = os.path.join(minor_save_dir, 'raw.csv')
    uniform_raw_path = os.path.join(uniform_save_dir, 'raw.csv')